<a href="https://colab.research.google.com/github/Farmhouse121/Adventures-in-Financial-Data-Science/blob/main/Book/Section%202.4%20Interest%20Rates/Section%202.4.1.4%20Testing%20for%20Independence%20in%20the%20Direction%20of%20Changes%20of%20Interest%20Rates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Installing arch...")
!pip install arch 1>/dev/null
import pandas as pd
import numpy as np ;
import matplotlib.pyplot as pl
from datetime import datetime
zero,one,two,three,five,ten,hundred=0e0,1e0,2e0,3e0,5e0,1e1,1e2 # some friendly numbers
half,GoldenRatio=one/two,(one+np.sqrt(five))/two

Installing arch...


In [ ]:
# get the bill rate data from Fred
DTB3=pd.read_table('https://fred.stlouisfed.org/data/DTB3.txt',skiprows=11,sep=r'\s+',header=0).rename(columns={"VALUE":"Rate","DATE":"Date"}).set_index("Date")
DTB3.index=pd.DatetimeIndex(DTB3.index).to_period('D')
DTB3['Rate']=DTB3['Rate'].apply(lambda x:float(x) if x!='.' else np.nan)

# join the recessions data (NBER data at Fred) 
USREC=pd.read_table('https://fred.stlouisfed.org/data/USREC.txt',skiprows=65,sep=r'\s+',header=0).rename(columns={"VALUE":"Recession","DATE":"Date"}).set_index("Date")
USREC.index=pd.DatetimeIndex(USREC.index).to_period('D')
DTB3=DTB3.merge(USREC,left_index=True,right_index=True,how='outer')
DTB3['Recession'].ffill(inplace=True)
DTB3['Recession']=DTB3['Recession'].apply(lambda x:True if x else False)
DTB3.dropna(inplace=True)
DTB3['Year']=DTB3.index.year
DTB3.index=pd.DatetimeIndex(DTB3.index.to_timestamp()).to_period('B') # business days
DTB3['Change']=DTB3['Rate']-DTB3['Rate'].shift().ffill() # changes that skip over holidays, why is this so hard?
DTB3.dropna(inplace=True)
display(DTB3)

,Rate,Recession,Year,Change
Date,,,,
1954-01-05,1.28,True,1954,-0.05
1954-01-06,1.28,True,1954,0.00
1954-01-07,1.31,True,1954,0.03
1954-01-08,1.31,True,1954,0.00
1954-01-11,1.33,True,1954,0.02
...,...,...,...,...
2023-03-28,4.64,False,2023,-0.11
2023-03-29,4.64,False,2023,0.00
2023-03-30,4.80,False,2023,0.16
